In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(dirname, len(filenames))
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import csv

## adding private key manually

In [ ]:
!gcloud auth activate-service-account \
  kaggle-automl-vision-sa@trusty-mantra-251121.iam.gserviceaccount.com \
          --key-file=/kaggle/input/private-yufengguo556835-gcp-key/key.json \
          --project=trusty-mantra-251121

In [ ]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('/kaggle/input/private-yufengguo556835-gcp-key/key.json')

In [ ]:
PROJECT_ID = 'trusty-mantra-251121'
REGION='us-central1'           # this MUST be us-central1
BUCKET_NAME='my-kaggle-data1'  # This bucket should already exist in us-central1

## Create AutoML Tables Client with manual credentials

In [ ]:
from google.cloud import automl_v1beta1 as automl
from google.cloud import storage

In [ ]:
storage_client = storage.Client(project=PROJECT_ID, credentials=credentials)
tables_gcs_client = automl.GcsClient(client=storage_client, bucket_name=BUCKET_NAME, credentials=credentials)


In [ ]:
tables_client = automl.TablesClient(project=PROJECT_ID, region=REGION, 
                                    gcs_client=tables_gcs_client, credentials=credentials)


## Create AutoML Tables client with Kaggle credentials

In [ ]:
from kaggle.gcp import KaggleKernelCredentials
from kaggle_secrets import GcpTarget
import kaggle_gcp

In [ ]:
os.environ['KAGGLE_USER_SECRETS_TOKEN']

In [ ]:
KaggleKernelCredentials(GcpTarget.AUTOML).valid

In [ ]:
storage_client = storage.Client(project=PROJECT_ID, credentials=KaggleKernelCredentials(GcpTarget.GCS))
tables_gcs_client = automl.GcsClient(client=storage_client, bucket_name=BUCKET_NAME, credentials=KaggleKernelCredentials(GcpTarget.GCS))


In [ ]:
tables_client = automl.TablesClient(project=PROJECT_ID, region=REGION, 
                                    gcs_client=tables_gcs_client, credentials=KaggleKernelCredentials(GcpTarget.AUTOML))

## (WORKAROUND) Create a Tables client with Kaggle credentials

In [ ]:
from google.cloud import storage, automl_v1beta1 as automl

storage_client = storage.Client(project=PROJECT_ID)
tables_gcs_client = automl.GcsClient(client=storage_client, bucket_name=BUCKET_NAME)
automl_client = automl.AutoMlClient()
prediction_client = automl.PredictionServiceClient()
tables_client = automl.TablesClient(project=PROJECT_ID, region=REGION, 
                                    client=automl_client, gcs_client=tables_gcs_client, prediction_client=prediction_client)

In [ ]:
# dataset_metadata = {"primary_table_spec_id": "label"} 
# # Set dataset name and metadata of the dataset.
# my_dataset = {
#     "tables_dataset_metadata": dataset_metadata,
# }

dataset_display_name='kaggle_data_tester'
my_dataset = tables_client.create_dataset(dataset_display_name=dataset_display_name)


### List datasets

In [ ]:
list_datasets = tables_client.list_datasets()
# Using a dict is problematic since display_name does not have to be unique
# datasets = { dataset.display_name: dataset.name for dataset in list_datasets }
datasets = [ (dataset.display_name, dataset.name) for dataset in list_datasets ]

datasets

### Upload file to GCS

In [ ]:
def upload_blob(source_file_name, destination_blob_name, bucket_name=BUCKET_NAME):
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    destination_gcs_uri = f'gs://{bucket_name}/{destination_blob_name}'
    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_gcs_uri))
    return destination_gcs_uri

In [ ]:
gcs_upload_uri = upload_blob('/kaggle/input/Kannada-MNIST/train.csv', 'kannada_train.csv')
gcs_upload_uri

### upload data to AutoML Tables

In [ ]:
import_data_operation = tables_client.import_data(
    dataset=my_dataset,
    gcs_input_uris=gcs_upload_uri
)
print('Dataset import operation: {}'.format(import_data_operation))

In [ ]:
# Synchronous check of operation status. Wait until import is done.
import_data_operation.result()
dataset = tables_client.get_dataset(dataset_name=my_dataset.name)
dataset

## Normal automl client, Auth with Kaggle Key

In [ ]:
automl_client = automl.AutoMlClient()

In [ ]:
project_location = automl_client.location_path(PROJECT_ID, REGION)
# Specify the image classification type for the dataset, MULTICLASS or MULTILABEL
dataset_metadata = {"primary_table_spec_id": "label"} 
# Set dataset name and metadata of the dataset.
my_dataset = {
    "display_name": 'kannada_kaggle',
    "tables_dataset_metadata": dataset_metadata,
}

# Create a dataset with the dataset metadata in the region.
# dataset = automl_client.create_dataset(project_location, my_dataset)

## Automl base client, with manual key

In [ ]:
automl_client = automl.AutoMlClient(credentials=credentials)

In [ ]:
project_location = automl_client.location_path(PROJECT_ID, REGION)
# Specify the image classification type for the dataset, MULTICLASS or MULTILABEL
dataset_metadata = {"primary_table_spec_id": "label"} 
# Set dataset name and metadata of the dataset.
my_dataset = {
    "display_name": 'kannadakaggle_manual',
    "tables_dataset_metadata": dataset_metadata,
}

# Create a dataset with the dataset metadata in the region.
# dataset = automl_client.create_dataset(project_location, my_dataset)

In [ ]:
data_train_file = "../input/Kannada-MNIST/train.csv"
data_test_file = "../input/Kannada-MNIST/test.csv" # test file for submission
data_dig_file = "../input/Kannada-MNIST/Dig-MNIST.csv" #labeled validation set

df_train = pd.read_csv(data_train_file)
df_test = pd.read_csv(data_test_file)
df_dig = pd.read_csv(data_dig_file)

In [ ]:
df_train.head()